# Self-RAG: A Dynamic Approach to RAG

## Setting Up the Environment

In [26]:
import numpy as np
import json
import fitz
import re

In [27]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = "gpt-4o-mini"

## Extracting Text from a PDF File

In [28]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일로부터 텍스트를 추출하는 함수

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        str: 추출된 전체 텍스트
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 전체 텍스트를 저장할 문자열 초기화

    # PDF의 각 페이지를 순회하며 텍스트 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 현재 페이지 가져오기
        text = page.get_text("text")  # 해당 페이지에서 텍스트 추출
        all_text += text  # 추출한 텍스트를 누적

    # 최종적으로 전체 텍스트 반환
    return all_text

## Chunking the Extracted Text

In [29]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 일정 길이로 나누되, 일부 겹치는 부분을 포함하여 청크로 분할하는 함수

    Args:
        text (str): 분할할 원본 텍스트
        n (int): 각 청크의 문자 수
        overlap (int): 청크 간 겹치는 문자 수

    Returns:
        List[str]: 분할된 텍스트 청크 리스트
    """
    chunks = []  # 분할된 청크들을 저장할 리스트 초기화

    # 시작 인덱스를 (n - overlap) 간격으로 이동하면서 청크 생성
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]  # i부터 i+n까지의 텍스트를 하나의 청크로 추출
        chunks.append(chunk)  # 추출한 청크를 리스트에 추가

    # 생성된 청크 리스트 반환
    return chunks

## Simple Vector Store Implementation

In [30]:
class SimpleVectorStore:
    """
    NumPy를 활용한 간단한 벡터 저장소 클래스
    텍스트, 임베딩, 메타데이터를 함께 관리하고 유사도 기반 검색을 지원
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []   # 임베딩 벡터들을 저장하는 리스트
        self.texts = []     # 원본 텍스트를 저장하는 리스트
        self.metadata = []  # 각 텍스트에 대한 메타데이터를 저장하는 리스트

    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 새로운 항목을 추가

        Args:
            text (str): 원본 텍스트
            embedding (List[float]): 해당 텍스트의 임베딩 벡터
            metadata (dict, optional): 텍스트와 관련된 부가 정보
        """
        self.vectors.append(np.array(embedding))        # 임베딩을 NumPy 배열로 변환하여 저장
        self.texts.append(text)                         # 원본 텍스트 저장
        self.metadata.append(metadata or {})            # 메타데이터가 없으면 빈 딕셔너리 저장

    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        주어진 질의 임베딩과 가장 유사한 항목 k개를 검색

        Args:
            query_embedding (List[float]): 질의 임베딩 벡터
            k (int): 반환할 유사 항목 수
            filter_func (callable, optional): 메타데이터 필터링 함수

        Returns:
            List[Dict]: 유사도가 높은 상위 k개의 항목 리스트 (텍스트, 메타데이터, 유사도 포함)
        """
        if not self.vectors:
            return []  # 저장된 벡터가 없으면 빈 리스트 반환

        # 질의 임베딩을 NumPy 배열로 변환
        query_vector = np.array(query_embedding)

        # 코사인 유사도를 기반으로 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 필터 함수가 있는 경우, 조건을 만족하지 않으면 건너뜀
            if filter_func and not filter_func(self.metadata[i]):
                continue

            # 코사인 유사도 계산
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 인덱스와 유사도 저장

        # 유사도 기준으로 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 k개의 결과를 구성하여 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],          # 관련 텍스트
                "metadata": self.metadata[idx],   # 관련 메타데이터
                "similarity": score               # 유사도 점수
            })

        return results  # 유사 항목 리스트 반환

## Creating Embeddings

In [31]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    주어진 텍스트에 대해 임베딩을 생성하는 함수

    Args:
        text (str 또는 List[str]): 임베딩을 생성할 입력 텍스트 또는 텍스트 리스트
        model (str): 임베딩 생성을 위한 모델 이름

    Returns:
        List[float] 또는 List[List[float]]: 생성된 임베딩 벡터 또는 벡터 리스트
    """
    # 입력이 문자열이면 리스트로 변환하여 처리
    input_text = text if isinstance(text, list) else [text]

    # 지정한 모델을 사용하여 임베딩 생성
    response = client.embeddings.create(
        model=model,
        input=input_text
    )

    # 단일 문자열 입력인 경우 첫 번째 임베딩만 반환
    if isinstance(text, str):
        return response.data[0].embedding

    # 리스트 입력인 경우 모든 임베딩 반환
    return [item.embedding for item in response.data]

## Document Processing Pipeline

In [32]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    Self-RAG을 위한 문서 처리 함수

    Args:
        pdf_path (str): PDF 파일 경로
        chunk_size (int): 각 청크의 문자 수
        chunk_overlap (int): 청크 간 겹치는 문자 수

    Returns:
        SimpleVectorStore: 문서 청크와 임베딩을 포함하는 벡터 저장소
    """
    # PDF 파일에서 텍스트 추출
    print("PDF에서 텍스트 추출 중...")
    extracted_text = extract_text_from_pdf(pdf_path)

    # 추출된 텍스트를 일정 길이로 분할
    print("텍스트 청크 분할 중...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"{len(chunks)}개의 텍스트 청크 생성 완료")

    # 각 청크에 대해 임베딩 생성
    print("청크 임베딩 생성 중...")
    chunk_embeddings = create_embeddings(chunks)

    # 벡터 저장소 초기화
    store = SimpleVectorStore()

    # 각 청크와 임베딩, 메타데이터를 저장소에 추가
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": pdf_path}
        )

    print(f"{len(chunks)}개의 청크가 벡터 저장소에 추가됨")

    # 벡터 저장소 반환
    return store

## Self-RAG Components
### 1. Retrieval Decision

In [33]:
def determine_if_retrieval_needed(query):
    """
    주어진 질의에 대해 외부 정보 검색이 필요한지 여부를 판단하는 함수

    Args:
        query (str): 사용자 질의

    Returns:
        bool: 검색이 필요한 경우 True, 그렇지 않으면 False
    """
    # AI에게 검색 필요 여부를 판별하는 기준을 알려주는 시스템 프롬프트
    system_prompt = """당신은 주어진 질의에 대해 검색이 필요한지를 판별하는 AI 어시스턴트입니다.
    사실 기반 질문, 특정 정보 요청, 사건, 인물, 개념에 대한 질문이라면 "Yes"라고 답하세요.
    의견, 가정적 시나리오, 일반 상식에 해당하는 질문은 "No"라고 답하세요.
    반드시 "Yes" 또는 "No"로만 답변하세요."""

    # 사용자 질의를 포함한 프롬프트 구성
    user_prompt = f"Query: {query}\n\nIs retrieval necessary to answer this query accurately?"

    # AI 모델 호출을 통해 검색 필요 여부 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 응답 내용에서 "yes" 또는 "no" 추출 후 소문자로 변환
    answer = response.choices[0].message.content.strip().lower()

    # "yes"가 포함되어 있으면 True 반환, 아니면 False
    return "yes" in answer

### 2. Relevance Evaluation

In [34]:
def evaluate_relevance(query, context):
    """
    주어진 문서 내용이 사용자 질의와 관련이 있는지를 평가하는 함수

    Args:
        query (str): 사용자 질의
        context (str): 문서 또는 텍스트 콘텐츠

    Returns:
        str: 'relevant' 또는 'irrelevant' 중 하나
    """
    # AI에게 문서 관련성 판단 기준을 안내하는 시스템 프롬프트
    system_prompt = """당신은 문서가 특정 질의와 관련이 있는지를 판단하는 AI 어시스턴트입니다.
    문서가 질의에 답하는 데 도움이 되는 정보를 포함하고 있는지를 기준으로 판단하세요.
    반드시 "Relevant" 또는 "Irrelevant" 중 하나로만 답변하세요."""

    # 너무 긴 문맥은 잘라서 처리 (토큰 초과 방지)
    max_context_length = 2000
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncated]"

    # 사용자 프롬프트 구성: 질의와 문서 내용 포함
    user_prompt = f"""Query: {query}
    Document content:
    {context}

    이 문서가 쿼리와 관련이 있나요? "관련 있음" 또는 "관련 없음"으로만 답변하세요.
    """

    # AI 모델 호출을 통해 관련성 판단 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 응답을 소문자로 변환하여 반환
    answer = response.choices[0].message.content.strip().lower()

    return answer

### 3. Support Assessment

In [35]:
def assess_support(response, context):
    """
    응답이 문맥에 의해 얼마나 잘 뒷받침되는지를 평가하는 함수

    Args:
        response (str): 생성된 응답
        context (str): 응답의 근거가 되는 문서 또는 텍스트

    Returns:
        str: 'fully supported', 'partially supported', 'no support' 중 하나
    """
    # 문맥이 응답을 뒷받침하는지를 판단하는 기준 안내용 시스템 프롬프트
    system_prompt = """당신은 주어진 문맥이 응답 내용을 얼마나 잘 뒷받침하는지를 평가하는 AI 어시스턴트입니다.
    응답 내 주장, 사실, 정보가 문맥에 의해 근거를 갖는지를 판단하세요.
    반드시 다음 중 하나로만 답변하세요:
    - "Fully supported": 응답의 모든 정보가 문맥에 명확하게 근거함
    - "Partially supported": 일부 정보는 문맥에 근거하지만 일부는 아님
    - "No support": 문맥에 전혀 근거가 없거나 모순되는 정보가 포함됨
    """

    # 문맥이 너무 길 경우 일부만 사용 (토큰 초과 방지)
    max_context_length = 2000
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncated]"

    # 사용자 프롬프트 구성: 문맥과 응답 내용을 포함
    user_prompt = f"""Context:
    {context}

    Response:
    {response}

    이 응답이 문맥에 의해 얼마나 잘 뒷받침되는지 평가하세요. 반드시 "Fully supported", "Partially supported", 또는 "No support" 중 하나로만 답변하세요.
    """

    # AI 모델 호출을 통해 평가 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 응답에서 평가 결과 추출 후 소문자로 변환
    answer = response.choices[0].message.content.strip().lower()

    return answer  # 평가 결과 반환

### 4. Utility Evaluation

In [36]:
def rate_utility(query, response):
    """
    질의에 대한 응답의 유용성을 평가하는 함수

    Args:
        query (str): 사용자 질의
        response (str): 생성된 응답

    Returns:
        int: 유용성 평가 점수 (1점부터 5점까지)
    """
    # AI에게 유용성 평가 기준을 안내하는 시스템 프롬프트
    system_prompt = """당신은 질의에 대한 응답의 유용성을 평가하는 AI 어시스턴트입니다.
    응답이 질의를 얼마나 잘 해결하는지, 정보의 완전성, 정확성, 실용성을 고려하세요.
    다음 기준에 따라 1점에서 5점 사이로 평가하세요:
    - 1: 전혀 유용하지 않음
    - 2: 거의 유용하지 않음
    - 3: 보통 수준으로 유용함
    - 4: 매우 유용함
    - 5: 탁월하게 유용함
    반드시 1~5 중 하나의 숫자만 답변하세요."""

    # 사용자 질의와 응답을 포함한 프롬프트 구성
    user_prompt = f"""Query: {query}
    Response:
    {response}

    위 응답의 유용성을 1점에서 5점 사이로 평가하세요:"""

    # AI 모델 호출을 통해 유용성 점수 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 생성된 응답에서 숫자 추출
    rating = response.choices[0].message.content.strip()

    # 응답 내에서 1~5 사이의 숫자만 추출
    rating_match = re.search(r'[1-5]', rating)
    if rating_match:
        return int(rating_match.group())  # 숫자만 정수로 변환하여 반환

    return 3  # 실패 시 중간값 3점 반환

## Response Generation

In [37]:
def generate_response(query, context=None):
    """
    질의와 선택적 문맥을 기반으로 응답을 생성하는 함수

    Args:
        query (str): 사용자 질의
        context (str, optional): 참고할 문맥 텍스트 (선택)

    Returns:
        str: 생성된 응답 텍스트
    """
    # AI에게 도움이 되는 응답을 생성하라고 안내하는 시스템 프롬프트
    system_prompt = """당신은 유용한 AI 어시스턴트입니다. 명확하고 정확하며 정보에 기반한 응답을 제공하세요."""

    # 문맥이 제공된 경우, 문맥을 포함하여 사용자 프롬프트 구성
    if context:
        user_prompt = f"""Context:
        {context}

        Query: {query}

        위 문맥을 기반으로 질의에 응답하세요."""
    else:
        # 문맥이 없는 경우, 질의만 포함
        user_prompt = f"""Query: {query}

        최선을 다해 질의에 응답하세요."""

    # AI 모델을 호출하여 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    # 응답 텍스트 추출 후 양쪽 공백 제거하여 반환
    return response.choices[0].message.content.strip()

## Complete Self-RAG Implementation

In [38]:
def self_rag(query, vector_store, top_k=3):
    """
    Self-RAG 전체 파이프라인을 수행하는 함수

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 문서 청크를 담고 있는 벡터 저장소
        top_k (int): 초기 검색 시 반환할 문서 수

    Returns:
        dict: 질의, 생성된 응답, 과정 중 수집된 평가 메트릭을 포함한 결과 딕셔너리
    """
    print(f"\nSelf-RAG 시작: {query}\n")

    # 1단계: 외부 검색이 필요한 질의인지 판단
    print("1단계: 검색 필요 여부 판단 중...")
    retrieval_needed = determine_if_retrieval_needed(query)
    print(f"검색 필요 여부: {retrieval_needed}")

    # Self-RAG 과정 중 측정할 메트릭 초기화
    metrics = {
        "retrieval_needed": retrieval_needed,
        "documents_retrieved": 0,
        "relevant_documents": 0,
        "response_support_ratings": [],
        "utility_ratings": []
    }

    best_response = None
    best_score = -1

    if retrieval_needed:
        # 2단계: 질의 임베딩을 기반으로 문서 검색
        print("\n2단계: 관련 문서 검색 중...")
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=top_k)
        metrics["documents_retrieved"] = len(results)
        print(f"{len(results)}개의 문서 검색됨")

        # 3단계: 검색된 문서들의 관련성 평가
        print("\n3단계: 문서 관련성 평가 중...")
        relevant_contexts = []

        for i, result in enumerate(results):
            context = result["text"]
            relevance = evaluate_relevance(query, context)
            print(f"문서 {i+1} 관련성: {relevance}")
            if relevance == "relevant":
                relevant_contexts.append(context)

        metrics["relevant_documents"] = len(relevant_contexts)
        print(f"관련 문서 수: {len(relevant_contexts)}")

        # 4단계: 관련 문서 각각에 대해 응답 생성 및 평가
        if relevant_contexts:
            print("\n4단계: 관련 문서 기반 응답 생성 및 평가 중...")
            for i, context in enumerate(relevant_contexts):
                print(f"\n문맥 {i+1}/{len(relevant_contexts)} 처리 중...")

                print("응답 생성 중...")
                response = generate_response(query, context)

                print("응답의 문맥 기반 근거 평가 중...")
                support_rating = assess_support(response, context)
                print(f"근거 평가: {support_rating}")
                metrics["response_support_ratings"].append(support_rating)

                print("응답 유용성 평가 중...")
                utility_rating = rate_utility(query, response)
                print(f"유용성 점수: {utility_rating}/5")
                metrics["utility_ratings"].append(utility_rating)

                # 응답의 전반적 점수 계산
                support_score = {
                    "fully supported": 3,
                    "partially supported": 1,
                    "no support": 0
                }.get(support_rating, 0)

                overall_score = support_score * 5 + utility_rating
                print(f"전체 점수: {overall_score}")

                # 가장 높은 점수의 응답 저장
                if overall_score > best_score:
                    best_response = response
                    best_score = overall_score
                    print("새로운 최적 응답 업데이트됨")

        # 관련 문서가 없거나 응답 품질이 낮은 경우
        if not relevant_contexts or best_score <= 0:
            print("\n적절한 문맥이 없거나 응답 품질이 낮아 문맥 없이 직접 생성 중...")
            best_response = generate_response(query)
    else:
        # 검색 없이 직접 응답 생성
        print("\n검색 없이 직접 응답 생성 중...")
        best_response = generate_response(query)

    # 최종 메트릭 정리
    metrics["best_score"] = best_score
    metrics["used_retrieval"] = retrieval_needed and best_score > 0

    print("\nSelf-RAG 완료")

    return {
        "query": query,
        "response": best_response,
        "metrics": metrics
    }

## Running the Complete Self-RAG System

In [39]:
def run_self_rag_example():
    """
    Self-RAG 시스템의 전체 작동 예시를 보여주는 함수
    """
    # 문서 전처리 수행
    pdf_path = "dataset/AI_Understanding.pdf"  # 처리할 PDF 문서 경로
    print(f"문서 처리 중: {pdf_path}")
    vector_store = process_document(pdf_path)  # 문서를 벡터 저장소로 변환

    # 예제 1: 검색이 필요할 가능성이 높은 질의
    query1 = "AI 개발의 주요 윤리적 문제는 무엇인가요?"
    print("\n" + "="*80)
    print(f"예제 1: {query1}")
    result1 = self_rag(query1, vector_store)  # Self-RAG 실행
    print("\n최종 응답:")
    print(result1["response"])  # 생성된 응답 출력
    print("\n메트릭:")
    print(json.dumps(result1["metrics"], indent=2))  # 평가 지표 출력

    # 예제 2: 검색 없이 직접 생성해도 되는 창작형 질의
    query2 = "인공지능에 대한 시 한 편을 쓸 수 있나요?"
    print("\n" + "="*80)
    print(f"예제 2: {query2}")
    result2 = self_rag(query2, vector_store)
    print("\n최종 응답:")
    print(result2["response"])
    print("\n메트릭:")
    print(json.dumps(result2["metrics"], indent=2))

    # 예제 3: 문서와 관련 있지만 추가 지식이 필요한 복합 질의
    query3 = "AI가 개발도상국의 경제 성장에 어떤 영향을 미칠까요?"
    print("\n" + "="*80)
    print(f"예제 3: {query3}")
    result3 = self_rag(query3, vector_store)
    print("\n최종 응답:")
    print(result3["response"])
    print("\n메트릭:")
    print(json.dumps(result3["metrics"], indent=2))

    # 결과 딕셔너리로 반환
    return {
        "example1": result1,
        "example2": result2,
        "example3": result3
    }

## Evaluating Self-RAG Against Traditional RAG

In [40]:
def traditional_rag(query, vector_store, top_k=3):
    """
    전통적인 RAG 방식으로 질의에 응답을 생성하는 함수 (비교용)

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 문서 청크를 포함한 벡터 저장소
        top_k (int): 검색할 문서 수

    Returns:
        str: 생성된 응답 텍스트
    """
    print(f"\n전통적인 RAG 실행 중: {query}\n")

    # 질의 임베딩 생성 및 유사 문서 검색
    print("문서 검색 중...")
    query_embedding = create_embeddings(query)
    results = vector_store.similarity_search(query_embedding, k=top_k)
    print(f"{len(results)}개의 문서 검색됨")

    # 검색된 문서들의 텍스트를 하나의 문맥으로 결합
    contexts = [result["text"] for result in results]
    combined_context = "\n\n".join(contexts)

    # 결합된 문맥을 기반으로 응답 생성
    print("응답 생성 중...")
    response = generate_response(query, combined_context)

    return response

In [41]:
def evaluate_rag_approaches(pdf_path, test_queries, reference_answers=None):
    """
    Self-RAG과 전통적인 RAG 방식을 비교 평가하는 함수

    Args:
        pdf_path (str): 문서 경로
        test_queries (List[str]): 테스트 질의 리스트
        reference_answers (List[str], optional): 평가용 참조 정답 리스트

    Returns:
        dict: 평가 결과 딕셔너리 (질의별 비교 결과 및 전체 분석 포함)
    """
    print("RAG 방식 비교 평가 시작")

    # 문서를 처리하여 벡터 저장소 생성
    vector_store = process_document(pdf_path)

    results = []

    for i, query in enumerate(test_queries):
        print(f"\n질의 {i+1} 처리 중: {query}")

        # Self-RAG 실행
        self_rag_result = self_rag(query, vector_store)
        self_rag_response = self_rag_result["response"]

        # 전통적인 RAG 실행
        trad_rag_response = traditional_rag(query, vector_store)

        # 참조 정답이 있다면 불러옴
        reference = reference_answers[i] if reference_answers and i < len(reference_answers) else None

        # 응답 비교 수행 (정량 또는 정성 평가)
        comparison = compare_responses(
            query,
            self_rag_response,
            trad_rag_response,
            reference
        )

        # 결과 저장
        results.append({
            "query": query,
            "self_rag_response": self_rag_response,
            "traditional_rag_response": trad_rag_response,
            "reference_answer": reference,
            "comparison": comparison,
            "self_rag_metrics": self_rag_result["metrics"]
        })

    # 전체 결과 분석 수행 (예: 점수 평균, 빈도, 요약 등)
    overall_analysis = generate_overall_analysis(results)

    return {
        "results": results,
        "overall_analysis": overall_analysis
    }

In [42]:
def compare_responses(query, self_rag_response, trad_rag_response, reference=None):
    """
    Self-RAG과 전통 RAG 응답을 비교 분석하는 함수

    Args:
        query (str): 사용자 질의
        self_rag_response (str): Self-RAG 응답
        trad_rag_response (str): 전통 RAG 응답
        reference (str, optional): 참조 정답 (사실 검증용)

    Returns:
        str: 응답 비교 분석 결과
    """
    # 시스템 프롬프트: 비교 기준과 역할 정의
    system_prompt = """당신은 RAG 시스템 응답 평가 전문가입니다.
    당신의 임무는 두 가지 RAG 접근 방식의 응답을 비교 분석하는 것입니다:

    1. Self-RAG: 검색 필요 여부를 동적으로 판단하고, 관련성과 응답 품질을 평가함
    2. 전통 RAG: 항상 문서를 검색하여 그 내용을 기반으로 응답을 생성함

    다음 기준에 따라 응답을 비교하세요:
    - 질의와의 관련성
    - 사실적 정확성
    - 정보의 완전성과 유익함
    - 간결성 및 초점의 명확성"""

    # 사용자 프롬프트 구성
    user_prompt = f"""질의:
    {query}

    Self-RAG 응답:
    {self_rag_response}

    전통 RAG 응답:
    {trad_rag_response}
    """

    # 참조 정답이 있다면 포함
    if reference:
        user_prompt += f"""
    참조 정답 (사실 검증용):
    {reference}
    """

    # 평가 요청 문구 추가
    user_prompt += """
    위 두 응답을 비교하고 어떤 응답이 더 나은지 그 이유를 설명하세요.
    정확성, 관련성, 정보의 완전성, 응답 품질을 중심으로 평가해주세요.
    """

    # LLM을 통해 비교 분석 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 평가에 적합한 모델 사용
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 분석 결과 텍스트 반환
    return response.choices[0].message.content

In [43]:
def generate_overall_analysis(results):
    """
    Self-RAG과 전통적인 RAG의 테스트 결과를 바탕으로 종합 분석을 생성하는 함수

    Args:
        results (List[Dict]): evaluate_rag_approaches에서 생성된 비교 결과 리스트

    Returns:
        str: 전체 분석 결과 텍스트
    """
    # LLM에게 비교 분석의 기준과 작성 방향을 안내하는 시스템 프롬프트
    system_prompt = """당신은 RAG 시스템 평가 전문가입니다. 여러 테스트 질의 결과를 바탕으로
    Self-RAG과 전통적인 RAG을 비교 분석하세요.

    다음 항목에 중점을 두어 분석을 작성하세요:
    1. Self-RAG이 더 잘 작동하는 경우와 그 이유
    2. 전통 RAG이 더 잘 작동하는 경우와 그 이유
    3. Self-RAG의 동적 검색 판단이 미치는 영향
    4. Self-RAG 내 관련성 및 근거 평가의 가치
    5. 질의 유형에 따른 접근 방식 선택에 대한 권장사항"""

    # 각 질의별 비교 결과 요약 텍스트 생성
    comparisons_summary = ""
    for i, result in enumerate(results):
        comparisons_summary += f"질의 {i+1}: {result['query']}\n"
        comparisons_summary += f"Self-RAG 메트릭: 검색 필요 여부: {result['self_rag_metrics']['retrieval_needed']}, "
        comparisons_summary += f"관련 문서 수: {result['self_rag_metrics']['relevant_documents']}/{result['self_rag_metrics']['documents_retrieved']}\n"
        comparisons_summary += f"비교 요약: {result['comparison'][:200]}...\n\n"

    # 사용자 프롬프트 구성: 비교 요약 전체 포함
    user_prompt = f"""다음은 총 {len(results)}개의 테스트 질의에 대한 Self-RAG vs 전통 RAG 비교 요약입니다:

    {comparisons_summary}

    이 요약을 기반으로 두 접근 방식에 대한 종합 분석을 작성해주세요."""

    # LLM 호출을 통해 분석 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 생성된 분석 결과 텍스트 반환
    return response.choices[0].message.content

## Evaluating the Self-RAG

In [44]:
# AI 정보 문서 경로
pdf_path = "dataset/AI_Understanding.pdf"

# Self-RAG의 적응형 검색 전략을 테스트하기 위한 다양한 유형의 테스트 질의 정의
test_queries = [
    "AI 개발에서 주요한 윤리적 문제는 무엇인가요?",  # 문서 기반 정보 질의
    # "설명 가능한 AI는 어떻게 신뢰를 향상시키나요?",  # 문서 기반 정보 질의
    # "인공지능에 관한 짧은 시를 써주세요",             # 창의적 질의 (검색 불필요)
    # "초지능 AI는 인간의 소외를 초래할까요?"           # 가설적 질의 (부분 검색 필요)
]

# 보다 객관적인 평가를 위한 참조 정답
reference_answers = [
    "AI 개발에서 주요한 윤리적 문제는 편향과 공정성, 프라이버시, 투명성, 책임성, 안전성, 악용 가능성 등입니다.",
    # "설명 가능한 AI는 의사결정 과정을 사용자에게 이해 가능하게 제공하여 공정성 검증, 편향 탐지, 시스템 신뢰 형성에 기여합니다.",
    # "양질의 인공지능 시는 AI의 가능성과 한계, 인간과의 관계, 미래 사회, 인식과 지능에 대한 철학적 탐구 등을 창의적으로 표현해야 합니다.",
    # "초지능 AI의 인간 소외에 대한 관점은 다양합니다. 일부는 경제적 대체나 통제 상실을 우려하며, 다른 일부는 보완적 역할과 인간 중심 설계로 여전히 인간이 중요하다고 봅니다. 대부분의 전문가는 안전하고 책임 있는 AI 설계가 핵심이라고 강조합니다."
]

# Self-RAG과 전통 RAG 접근법을 비교 평가 실행
evaluation_results = evaluate_rag_approaches(
    pdf_path=pdf_path,                  # AI 정보를 담고 있는 문서 경로
    test_queries=test_queries,          # AI 관련 테스트 질의 리스트
    reference_answers=reference_answers # 평가용 기준 정답
)

# 최종 종합 비교 분석 결과 출력
print("\n***전체 비교 분석 결과***\n")
print(evaluation_results["overall_analysis"])

RAG 방식 비교 평가 시작
PDF에서 텍스트 추출 중...
텍스트 청크 분할 중...
21개의 텍스트 청크 생성 완료
청크 임베딩 생성 중...
21개의 청크가 벡터 저장소에 추가됨

질의 1 처리 중: AI 개발에서 주요한 윤리적 문제는 무엇인가요?

Self-RAG 시작: AI 개발에서 주요한 윤리적 문제는 무엇인가요?

1단계: 검색 필요 여부 판단 중...
검색 필요 여부: True

2단계: 관련 문서 검색 중...
3개의 문서 검색됨

3단계: 문서 관련성 평가 중...
문서 1 관련성: relevant
문서 2 관련성: relevant
문서 3 관련성: relevant
관련 문서 수: 3

4단계: 관련 문서 기반 응답 생성 및 평가 중...

문맥 1/3 처리 중...
응답 생성 중...
응답의 문맥 기반 근거 평가 중...
근거 평가: fully supported
응답 유용성 평가 중...
유용성 점수: 5/5
전체 점수: 20
새로운 최적 응답 업데이트됨

문맥 2/3 처리 중...
응답 생성 중...
응답의 문맥 기반 근거 평가 중...
근거 평가: fully supported
응답 유용성 평가 중...
유용성 점수: 5/5
전체 점수: 20

문맥 3/3 처리 중...
응답 생성 중...
응답의 문맥 기반 근거 평가 중...
근거 평가: fully supported
응답 유용성 평가 중...
유용성 점수: 5/5
전체 점수: 20

Self-RAG 완료

전통적인 RAG 실행 중: AI 개발에서 주요한 윤리적 문제는 무엇인가요?

문서 검색 중...
3개의 문서 검색됨
응답 생성 중...

***전체 비교 분석 결과***

### Self-RAG vs 전통 RAG: 종합 분석

#### 1. Self-RAG이 더 잘 작동하는 경우와 그 이유
Self-RAG은 특정 질의에 대해 더 높은 관련성과 정확성을 제공하는 경우가 많습니다. 예를 들어, "AI 개발에서 주요한 윤리적 문제는 무엇인가요?"라는 질의에 대해 Self-RAG은 직접적인 답